In [3]:
import os

from dotenv import find_dotenv, load_dotenv
import pdfplumber
from telegram import Bot, Update
from telegram.ext import CommandHandler, Filters, MessageHandler, Updater

# Load environment variables
_ = load_dotenv(find_dotenv()) # read local .env file

# Set up telegram's bot token
BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")

# Set up owner chat id
OWNER_CHAT_ID = os.getenv("OWNER_CHAT_ID")

In [7]:
def start(update: Update, context: Bot) -> None:
    """Send a message when the command /start is issued."""

    # Greet the user and introduce the bot
    update.message.reply_text(
        """\
Hello! I am Resume Reviewer Bot. I am here to help you with your resume. \
Simply send me your resume in PDF format and I will review it for you. I shall \
provide you with feedback on how you can improve your resume.\
"""
    )


def forward_convo_to_owner(
    update: Update,
    user_text: str,
    bot_response: str,
    context: Bot
) -> None:
    """
    Forward the conversations to the owner of the bot in real-time like logs
    """

    # Forward the conversation to the owner of the bot
    user_firstName = update.message.from_user.first_name
    conversation = f"""\
********************************
{user_firstName}: {user_text}

ResumeReviewer_Bot: {bot_response}
********************************\
"""
    
    context.bot.send_message(chat_id=OWNER_CHAT_ID, text=conversation)

    
def extract_text_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()

    return text
    
def handle_pdf(update, context):
    file_id = update.message.document.file_id
    
    file = context.bot.get_file(file_id)
    
    file.download('input_resume.pdf')
    
    text = extract_text_from_pdf('input_resume.pdf')
    
    update.message.reply_text(text)

In [8]:
def main():
    updater = Updater(token=BOT_TOKEN, use_context=True)
    dp = updater.dispatcher

    # Handlers
    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(MessageHandler(Filters.document.mime_type("application/pdf"), handle_pdf))
    
    # Print a message to the terminal to indicate that the bot is starting
    print("ResumeReviewer_Bot is now running and listening for updates...\n\n")
    
    # Start the bot
    updater.start_polling()
    updater.idle()

In [9]:
main()

ResumeReviewer_Bot is now running and listening for updates...


